## RAG using Amazon Bedrock & Amazon OpenSearch

In [ ]:
%%capture
!./download-beta-sdk.sh
!pip install -r requirements.txt

In [ ]:
import os, sys
from utils import secret, opensearch, dataset, bedrock, iam

In [ ]:
#restore variables from the previous notebook
%store -r \
region endpoint_url opensearch_client opensearch_username opensearch_secret opensearch_index

### Initialize OpenSearch

In [ ]:
exists = opensearch.check_opensearch_index(opensearch_client, opensearch_index)  
if exists:
    print(f'OpenSearch index: {opensearch_index} was found')

### Connect to Amazon Bedrock

In [ ]:
def get_bedrock_client(region, url):
    module_path = "."
    sys.path.append(os.path.abspath(module_path))
    boto3_bedrock = bedrock.get_bedrock_client(
        region = region,
        url_override = url
    )
    return boto3_bedrock

In [ ]:
endpoint_url = 'https://prod.us-west-2.frontend.bedrock.aws.dev'
bedrock_client = get_bedrock_client(region,endpoint_url)
bedrock_client.list_foundation_models()

In [ ]:
from langchain.llms.bedrock import Bedrock
from langchain.embeddings import BedrockEmbeddings
from langchain.vectorstores import OpenSearchVectorSearch

def create_bedrock_llm(bedrock_client):
    bedrock_llm = Bedrock(
        model_id="amazon.titan-tg1-large", 
        client=bedrock_client,
        model_kwargs={'temperature': 0}
        )
    return bedrock_llm


def get_bedrock_embeddings_client(bedrock_client):
    bedrock_embeddings_client = BedrockEmbeddings(
        client=bedrock_client,
        model_id="amazon.titan-e1t-medium"
    )
    return bedrock_embeddings_client


def get_opensearch_vector_search_client(index_name, opensearch_password, bedrock_embeddings_client, opensearch_endpoint, _is_aoss=False):
    vector_search = OpenSearchVectorSearch(
        index_name=index_name,
        embedding_function=bedrock_embeddings_client,
        opensearch_url=f"https://{opensearch_endpoint}",
        http_auth=(index_name, opensearch_password),
        is_aoss=_is_aoss
    )
    return vector_search

In [ ]:
bedrock_client = get_bedrock_client(region, endpoint_url)
bedrock_llm = create_bedrock_llm(bedrock_client)
bedrock_embeddings_client = get_bedrock_embeddings_client(bedrock_client)

In [ ]:
opensearch_endpoint = opensearch.get_opensearch_endpoint(opensearch_index, region)
opensearch_vector_search_client = get_opensearch_vector_search_client(opensearch_index, opensearch_secret, bedrock_embeddings_client, opensearch_endpoint)

## Perform RAG

In [ ]:
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA

In [ ]:
question = "are the babies' eye ball same as adults'?"

In [ ]:
prompt_template = """Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer. don't include harmful content

{context}

Question: {question}
Answer:"""
PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)

In [ ]:
qa = RetrievalQA.from_chain_type(llm=bedrock_llm, 
                                     chain_type="stuff", 
                                     retriever=opensearch_vector_search_client.as_retriever(),
                                     return_source_documents=True,
                                     chain_type_kwargs={"prompt": PROMPT, "verbose": True},
                                     verbose=True)

In [ ]:
response = qa(question, return_only_outputs=True)
documents = response.get('source_documents')

In [ ]:
print(f'Retrieval from OpenSearch matching the question: \"{question}\"')
for doc in documents:
    print(f'{doc.page_content}')
    
print(f"\nAugmented response from Titan titan-tg1-large LLM:\n{response.get('result')}")